In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [47]:
#vocab_size = 4000
#max_len_list = 1119

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nv

In [18]:
from datasets import load_dataset

dataset = load_dataset("roneneldan/TinyStories")

In [23]:
len(dataset["validation"]['text'])

21990

In [24]:
dataset = dataset["validation"]['text']

In [25]:
dataset2 = []
for i in dataset:
    if len(i) > 100:
        dataset2.append(i)

In [30]:
len(dataset2)
dataset2[:2]

['Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright and clean!" Kitty smiled and replied, "Thank you, Spot. I polish it every day."\n\nAfter playing with the car, Kitty and Spot felt thirsty. They found a small pond with clear water. They drank the water and felt very happy. They played together all day and became best friends.',
 'Once upon a time, in a big forest, there lived a rhinoceros named Roxy. Roxy loved to climb. She climbed trees, rocks, and hills. One day, Roxy found an icy hill. She had never seen anything like it before. It was shiny and cold, and she wanted to climb it.\n\nRoxy tried to climb the icy hill, but it was very slippery. She tried again and again, but she kept falling down. Roxy was sad. She wanted to climb the icy hill so much. Then, she saw a little bird named Billy. Billy saw that Roxy was sad and asked, "Why are you sad, Roxy?"\n\nRoxy told Billy about the icy hill and how she couldn\'t climb it. Billy said, "I have an idea! Let\'s 

In [16]:
!pip install tokenizers

In [27]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, decoders

# 1. Make a BPE tokenizer
tokenizer = Tokenizer(models.BPE())

# 2. Pre-tokenizer: ByteLevel is common for GPT-like BPE
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

# 3. Decoder: ByteLevel to decode back
tokenizer.decoder = decoders.ByteLevel()

# 4. Trainer: define vocab size, min frequency, special tokens
trainer = trainers.BpeTrainer(
    vocab_size=4000,  # adjust as needed
    min_frequency=2,
    special_tokens=["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
)

# 5. Train on your list of strings
tokenizer.train_from_iterator(dataset2, trainer)

# 6. Save the tokenizer if you want
tokenizer.save("bpe_tokenizer.json")


In [28]:
print("Actual vocab size:", tokenizer.get_vocab_size())

Actual vocab size: 4000


In [ ]:
for sentence in dataset2:
    encoded = tokenizer.encode(sentence)
    print(f"Sentence: {sentence}")
    print(f"Tokens: {encoded.tokens}")
    print(f"IDs: {encoded.ids}")


In [36]:
encoded = tokenizer.encode(dataset2[0])
print(f"Sentence: {dataset2[0]}")
print(f"Tokens: {encoded.tokens}")
print(f"IDs: {encoded.ids}")

Sentence: Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright and clean!" Kitty smiled and replied, "Thank you, Spot. I polish it every day."

After playing with the car, Kitty and Spot felt thirsty. They found a small pond with clear water. They drank the water and felt very happy. They played together all day and became best friends.
Tokens: ['ĠSpot', '.', 'ĠSpot', 'Ġsaw', 'Ġthe', 'Ġshiny', 'Ġcar', 'Ġand', 'Ġsaid', ',', 'Ġ"', 'Wow', ',', 'ĠKitty', ',', 'Ġyour', 'Ġcar', 'Ġis', 'Ġso', 'Ġbright', 'Ġand', 'Ġclean', '!"', 'ĠKitty', 'Ġsmiled', 'Ġand', 'Ġreplied', ',', 'Ġ"', 'Thank', 'Ġyou', ',', 'ĠSpot', '.', 'ĠI', 'Ġpol', 'ish', 'Ġit', 'Ġevery', 'Ġday', '."', 'Ċ', 'Ċ', 'After', 'Ġplaying', 'Ġwith', 'Ġthe', 'Ġcar', ',', 'ĠKitty', 'Ġand', 'ĠSpot', 'Ġfelt', 'Ġthirsty', '.', 'ĠThey', 'Ġfound', 'Ġa', 'Ġsmall', 'Ġpond', 'Ġwith', 'Ġclear', 'Ġwater', '.', 'ĠThey', 'Ġdrank', 'Ġthe', 'Ġwater', 'Ġand', 'Ġfelt', 'Ġvery', 'Ġhappy', '.', 'ĠThey', 'Ġplayed', 'Ġtogether', 'Ġall', 'Ġ

In [39]:
encoded_dataset = []
for sentence in dataset2:
    encoded = tokenizer.encode(sentence)
    encoded_dataset.append(encoded.ids)

In [46]:
max_len_list =0

for sequence in encoded_dataset:
  if len(sequence) >= max_len_list:
      max_len_list= len(sequence)

max_len_list

1119

In [51]:
print(encoded_dataset[0:2])

[[999, 16, 999, 264, 116, 823, 429, 118, 180, 14, 189, 1215, 14, 2456, 14, 508, 429, 275, 200, 966, 118, 796, 289, 2456, 359, 118, 929, 14, 189, 906, 209, 14, 999, 16, 194, 1434, 572, 157, 472, 207, 278, 99, 99, 1029, 462, 195, 116, 429, 14, 2456, 118, 999, 357, 3421, 16, 177, 434, 112, 647, 1144, 195, 2917, 615, 16, 177, 2776, 116, 615, 118, 357, 241, 258, 16, 177, 609, 414, 327, 207, 118, 955, 694, 335, 16], [276, 292, 112, 242, 14, 170, 112, 260, 891, 14, 245, 762, 112, 3831, 341, 962, 72, 3817, 16, 962, 72, 3817, 348, 119, 860, 16, 166, 1389, 1204, 14, 1800, 14, 118, 1274, 76, 16, 328, 207, 14, 962, 72, 3817, 434, 267, 3164, 1274, 16, 166, 214, 599, 1160, 1293, 378, 157, 829, 16, 265, 133, 823, 118, 1227, 14, 118, 188, 295, 119, 860, 157, 16, 99, 99, 49, 72, 3817, 582, 119, 860, 116, 3164, 1274, 14, 261, 157, 133, 241, 568, 2821, 82, 16, 166, 582, 437, 118, 437, 14, 261, 188, 781, 3459, 480, 16, 962, 72, 3817, 133, 421, 16, 166, 295, 119, 860, 116, 3164, 1274, 200, 564, 16, 728, 14

In [53]:
merged_dataset = []
for sentence in encoded_dataset:
    merged_dataset.extend(sentence)

In [58]:
import torch

class GPTChunkedDataset(torch.utils.data.Dataset):
    def __init__(self, token_ids, block_size):
        self.token_ids = token_ids
        self.block_size = block_size

        # How many full non-overlapping blocks fit?
        self.num_chunks = (len(self.token_ids) - 1) // block_size

    def __len__(self):
        return self.num_chunks

    def __getitem__(self, idx):
        start = idx * self.block_size
        x = torch.tensor(self.token_ids[start : start + self.block_size], dtype=torch.long)
        y = torch.tensor(self.token_ids[start + 1 : start + 1 + self.block_size], dtype=torch.long)
        return x, y


In [73]:
dataset = GPTChunkedDataset(merged_dataset, block_size=64)

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,   # Shuffle for training, optional
    drop_last=True,  # Drop partial batch
    pin_memory=True
)

In [74]:
for xb, yb in loader:
    print(xb.shape)  # (256, 64)     sequence_length=64
    print(yb.shape)  # (256, 64)     sequence_length=64
    break

torch.Size([256, 64])
torch.Size([256, 64])


In [75]:
import torch
import torch.nn as nn

class TokenAndPositionEmbedding(nn.Module):
    def __init__(self, vocab_size=4000, embed_dim=128, max_len=200):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_dim)
        self.position_emb = nn.Embedding(max_len, embed_dim)

    def forward(self, x):
        batch_size, seq_len = x.size()
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        token_embedding = self.token_emb(x)
        position_embedding = self.position_emb(positions)
        return token_embedding + position_embedding           #torch.Size([256, 64, 128])    b,s,e

In [76]:
tpe = TokenAndPositionEmbedding(
    vocab_size=4000,
    embed_dim=128,
    max_len=200
)

out = tpe(xb)
out.shape

torch.Size([256, 64, 128])

In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden_dim=None, dropout=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        
        self.attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.ln1 = nn.LayerNorm(embed_dim)

        ff_hidden_dim = ff_hidden_dim or embed_dim * 4
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden_dim),
            nn.ReLU(),
            nn.Linear(ff_hidden_dim, embed_dim)
        )
        self.ln2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        x: (batch_size, seq_len, embed_dim)
        """
        batch_size, seq_len, _ = x.size()

        # Transpose for MultiheadAttention: (seq_len, batch_size, embed_dim)
        x_t = x.transpose(0, 1)

        # Make causal mask: shape (seq_len, seq_len)
        attn_mask = torch.triu(torch.ones(seq_len, seq_len, device=x.device), diagonal=1).bool()

        # Masked self-attention
        attn_out, _ = self.attn(x_t, x_t, x_t, attn_mask=attn_mask)

        # Residual + LayerNorm
        x2 = self.ln1(x_t + self.dropout(attn_out))

        # Feed Forward
        ff_out = self.ff(x2)

        # Residual + LayerNorm
        out = self.ln2(x2 + self.dropout(ff_out))

        # Transpose back: (batch_size, seq_len, embed_dim)
        return out.transpose(0, 1)

In [78]:
import torch
import torch.nn as nn

class MiniGPTBlock(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=8, max_len=66):
        super().__init__()
        self.embedding = TokenAndPositionEmbedding(vocab_size, embed_dim, max_len)
        self.transformer = TransformerBlock(embed_dim, num_heads)
        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)  # (batch_size, seq_len, embed_dim)
        x = self.transformer(x)  # (batch_size, seq_len, embed_dim)
        logits = self.lm_head(x)  # (batch_size, vocab_size)
        return logits

In [79]:
model = MiniGPTBlock(vocab_size=4000)

In [80]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [81]:
model.to(device)

MiniGPTBlock(
  (embedding): TokenAndPositionEmbedding(
    (token_emb): Embedding(4000, 128)
    (position_emb): Embedding(66, 128)
  )
  (transformer): TransformerBlock(
    (attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
    (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (ff): Sequential(
      (0): Linear(in_features=128, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=128, bias=True)
    )
    (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (lm_head): Linear(in_features=128, out_features=4000, bias=True)
)

In [82]:
epochs = 50
learning_rate = 0.005

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [84]:
for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in loader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    logits = model(batch_x)  # (batch, seq_len, vocab_size)

    loss = criterion(
      logits.view(-1, vocab_size),
      batch_y.view(-1)
    )

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")


Epoch: 1, Loss: 1172.3973
Epoch: 2, Loss: 928.6028
Epoch: 3, Loss: 877.1469
Epoch: 4, Loss: 849.3757
Epoch: 5, Loss: 830.2931
Epoch: 6, Loss: 816.7812
Epoch: 7, Loss: 806.2519
Epoch: 8, Loss: 798.2785
Epoch: 9, Loss: 791.2130
Epoch: 10, Loss: 785.5917
Epoch: 11, Loss: 780.6116
Epoch: 12, Loss: 776.4056
Epoch: 13, Loss: 772.6126
Epoch: 14, Loss: 769.3013
Epoch: 15, Loss: 766.1249
Epoch: 16, Loss: 763.4112
Epoch: 17, Loss: 760.9264
Epoch: 18, Loss: 758.5742
Epoch: 19, Loss: 756.5006
Epoch: 20, Loss: 754.5983
Epoch: 21, Loss: 752.7565
Epoch: 22, Loss: 750.9802
Epoch: 23, Loss: 749.4719
Epoch: 24, Loss: 747.8123
Epoch: 25, Loss: 746.4548
Epoch: 26, Loss: 745.2027
Epoch: 27, Loss: 743.8756
Epoch: 28, Loss: 742.6226
Epoch: 29, Loss: 741.5002
Epoch: 30, Loss: 740.5686
Epoch: 31, Loss: 739.2706
Epoch: 32, Loss: 738.3365
Epoch: 33, Loss: 737.4951
Epoch: 34, Loss: 736.6225
Epoch: 35, Loss: 735.8308
Epoch: 36, Loss: 734.8306
Epoch: 37, Loss: 734.2506
Epoch: 38, Loss: 733.3519
Epoch: 39, Loss: 732

In [85]:
torch.save(model.state_dict(), 'gpt_from_scratch_in_pytorch_on_stories_generation.pth')

In [92]:
import torch

def generate(
    model,
    tokenizer,           # your tokenizer to decode IDs to text
    device,
    start_tokens,        # list of ints, e.g. [BOS] or your prompt encoded
    max_new_tokens=50,   # how many tokens to generate
    block_size=64,       # context window
    temperature=1.0,     # controls randomness
    top_k=None           # optional: restrict to top-k for more randomness control
):
    model.eval()
    generated = start_tokens.copy()
    input_ids = torch.tensor(start_tokens, dtype=torch.long).unsqueeze(0).to(device)  # (1, len)

    for _ in range(max_new_tokens):
        # If context is longer than block_size, crop oldest tokens
        if input_ids.size(1) > block_size:
            input_ids = input_ids[:, -block_size:]

        with torch.no_grad():
            logits = model(input_ids)  # (1, seq_len, vocab_size)

        # Get logits for last token only
        logits = logits[:, -1, :] / temperature  # (1, vocab_size)

        # Optionally top-k filter
        if top_k is not None:
            v, ix = torch.topk(logits, top_k)
            logits[logits < v[0, -1]] = -float('Inf')

        probs = torch.softmax(logits, dim=-1)  # (1, vocab_size)

        next_token = torch.multinomial(probs, num_samples=1)  # (1, 1)

        next_token_id = next_token.item()

        # Append to sequence
        generated.append(next_token_id)

        # Update input_ids
        input_ids = torch.cat([input_ids, next_token], dim=1)  # (1, seq_len+1)


    return generated


In [94]:
# Suppose your tokenizer encodes/decodes:
start_text = "Once upon a midnight dreary"
start_tokens = tokenizer.encode(start_text).ids  # Or whatever you used

output_tokens = generate(
    model=model,
    tokenizer=tokenizer,
    device=device,
    start_tokens=start_tokens,
    max_new_tokens=100,
    block_size=64,
    temperature=1.5,
    top_k=50  # try top_k for more diverse output
)

# Decode
output_text = tokenizer.decode(output_tokens)
print(output_text)


 Once upon a midnight dreary.  He lived in an organized. Every day - he wanted something wonderful and learn a valuable lesson â€”itating in the trees, trees, with them.

The vimp replied, "I have some to pick this peas."

Carikie thought for a moment every moment and then went on an adventure and went over it. Finally, there looked brighter, brighter for the lightning, no more teff. Paul wasn't alone anymore. 



In [95]:
tokenizer.save("my_tokenizer.json")